In [13]:
import numpy as np
import sklearn.gaussian_process as gpr
import time
from joblib import Parallel, delayed

import sys

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', RuntimeWarning)
warnings.simplefilter('ignore', ConvergenceWarning)
from scipy.optimize import minimize, Bounds, NonlinearConstraint
from collections import OrderedDict
import torch

from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch.optim import LBFGS

NameError: name '_C' is not defined

In [7]:
# Define Parameters
exp_w = 2.6
#C0varf = np.loadtxt('')
FR = np.array([0.100, 0.075, 0.075])
R_Frac = 1e-6
ub = np.array([423, 423])
lb = np.array([303, 303])
bounds = Bounds((0, 0), (1, 1))
dim = len(ub)
kernel = gpr.kernels.Matern((1, 1), ((0.06, 5), (0.06, 5)), nu = 2.5)

In [8]:
trialspltvar = 2

In [10]:
 timespltvar= np.zeros(trialspltvar+1)
timespltvar

array([0., 0., 0.])

In [12]:
time_vp = np.zeros(trialspltvar)
time_vp

array([0., 0.])

In [10]:
class LCB_AF():
    def __init__(self, model, dim, exp_w, descale, refmod = None, args = ()):
        self.model = model
        self.dim = dim
        self.exp_w = exp_w
        self.descale = descale
        self.args = args
        
        if refmod:
            self.refmod = refmod
        else:
            def zr(x):
                return 0
            self.refmod = zr
            
    def LCB(self, x):
        x = np.array([x]).reshape(-1,1);
        x = x.reshape(int(x.shape[0]/self.dim), self.dim)
        
        mu, std = self.model.predict(x, return_std=True);
        mu = mu.flatten()
        
        yref = self.refmod(self.descale(x), *self.args)
            
        return (yref+mu-self.exp_w*std).flatten()

In [8]:
# Set system and reference functions
SYST_RECYCLE = SYST_RECYCLE
SYST_RECYCLE_REF = RXTR_REG()


# Train reference model (GP)

TT = np.arange(303, 424, 1)
TT = np.meshgrid(TT, TT)
TT = np.hstack([TT[0].reshape(-1, 1), TT[1].reshape(-1, 1)])

print('Create Statistical Reference Model...')
start = time.time()

Tmod = np.linspace(303, 423, 13)
Tmod = np.meshgrid(Tmod, Tmod)
Tmod = np.hstack([Tmod[0].reshape(-1, 1), Tmod[1].reshape(-1, 1)])
CTREF = np.ones((Tmod.shape[0], 3))

Ctref = Parallel(n_jobs = 5)(delayed(SYST_RECYCLE_REF)(Tmod, FR, R_Frac, Cdist)
                             for Cdist in C0varf)

for i in range(3):
    C = np.vstack(Ctref[:][:]).T[:, i::3]
    C = 8*np.sum(C, axis = 1)
    CTREF[:, i] = C

kergp = gpr.kernels.Matern((5, 5), ((1, 10), (1, 10)), nu = 2.5)
gprefmod = gpr.GaussianProcessRegressor(kernel = kergp,
                                        alpha = 1e-6,
                                        n_restarts_optimizer = 10,
                                        normalize_y = True)
gprefmod.fit(scale(Tmod), CTREF)

end = time.time()
mobdtm = end-start
print(mobdtm)

def SYST_C(T):
    CtR = 0
    for i in range(C0varf.shape[0]):
        Ctr = 8*SYST_RECYCLE(T, FR, R_Frac, C0varf[i])[-1]
        CtR += Ctr
    return CtR

def SYST_C_DIST(T):
    CtR = 0
    for i in range(C0varf.shape[0]):
        Ctr = SYST_RECYCLE(T, FR, R_Frac, C0varf[i])
        Ctr = np.vstack(Ctr[:]).T
        CtR += 8*Ctr
    return CtR

def SYST_C_REFGP(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)[:, -1]

def SYST_C_DISTGP(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)

def SYST_C_REF1(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)[:, 0]

def SYST_C_REF2(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)[:, 1]

def zr(x):
    x = x.reshape(-1, 1)
    x = x.reshape(int(x.shape[0]/2), 2)
    return np.zeros((x.shape[0], 3))


# Setup BO class
REACOPTIM = BO_algos.BO(ub = ub,
                        lb = lb,
                        dim = 2,
                        exp_w = exp_w,
                        kernel = kernel,
                        system = SYST_C,
                        bounds = bounds,
                        **{'refmod': SYST_C_REFGP,
                           'distmod': SYST_C_DIST,
                           'ref_distmod': SYST_C_DISTGP,
                           'ref_distmod1': SYST_C_REF1,
                           'ref_distmod2': SYST_C_REF2})


## Generate level sets
gpparts = gpr.GaussianProcessRegressor(kernel = kergp,
                                       alpha = 1e-6,
                                       n_restarts_optimizer = 10,
                                       normalize_y = True)
gpparts.fit(scale(Tmod), (CTREF[:, -1]).reshape(-1, 1))

parts = np.array([-461, -383]) # ref mod with g_1(T_1) and g_2(T_1, T_2)

con1 = lambda x: (gpparts.predict(x.reshape(1, 2))).flatten()
con2 = lambda x: x[0]

nlc1 = NonlinearConstraint(con1, -1e4, parts[0])
nlc21 = NonlinearConstraint(con1, parts[0], parts[1])
nlc22 = NonlinearConstraint(con2, 0.4, 1.1)
nlc31 = NonlinearConstraint(con1, parts[0], parts[1])
nlc32 = NonlinearConstraint(con2, 0, 0.38)
nlc4 = NonlinearConstraint(con1, parts[1], 1e4)

cons = {'1': [nlc1], '2': [nlc21, nlc22], '3': [nlc31, nlc32], '4': [nlc4]}

## VP-BO intial points
lim_init = REACOPTIM.scale(np.array([336, 380]))


# Run_BO
trials_seq = 100
trials_par1 = 25
trials_par2 = 33

x_init = np.linspace(bounds.lb, bounds.ub, 5)
x_init = np.meshgrid(*x_init.T)
x_init = np.reshape(x_init, (dim, -1)).T

MET_SBO = np.zeros((trials_seq, 4))
PARAMS_SBO = np.ones(x_init.shape)
DIST_SBO = np.array([]).reshape(0, dim)
RES_SBO = np.ones((MET_SBO.shape[0], x_init.shape[0]))

MET_REFBO = np.zeros((trials_seq, 4))
PARAMS_REFBO = np.ones(x_init.shape)
DIST_REFBO = np.array([]).reshape(0, dim)
RES_REFBO = np.ones((MET_REFBO.shape[0], x_init.shape[0]))

MET_LSBO1 = np.zeros((trials_par1, 4))
PARAMS_LSBO1 = np.ones(x_init.shape)
DIST_LSBO1 = np.array([]).reshape(0, dim)
RES_LSBO1 = np.ones((MET_LSBO1.shape[0], x_init.shape[0]))

MET_LSBO2 = np.zeros((trials_par1, 4))
PARAMS_LSBO2 = np.ones(x_init.shape)
DIST_LSBO2 = np.array([]).reshape(0, dim)
RES_LSBO2 = np.ones((MET_LSBO2.shape[0], x_init.shape[0]))

MET_VPBO1 = np.zeros((trials_par2, 4))
PARAMS_VPBO1 = np.ones(x_init.shape)
DIST_VPBO1 = np.array([]).reshape(0, dim)
RES_VPBO1 = np.ones((MET_VPBO1.shape[0], x_init.shape[0]))

MET_VPBO2 = np.zeros((trials_par2, 4))
PARAMS_VPBO2 = np.ones(x_init.shape)
DIST_VPBO2 = np.array([]).reshape(0, dim)
RES_VPBO2 = np.ones((MET_VPBO2.shape[0], x_init.shape[0]))

MET_HSBO = np.zeros((trials_par1, 4))
PARAMS_HSBO = np.ones(x_init.shape)
DIST_HSBO = np.array([]).reshape(0, dim)
RES_HSBO = np.ones((MET_HSBO.shape[0], x_init.shape[0]))

MET_EXBO = np.zeros((trials_par1, 4))
PARAMS_EXBO = np.ones(x_init.shape)
DIST_EXBO = np.array([]).reshape(0, dim)
RES_EXBO = np.ones((MET_EXBO.shape[0], x_init.shape[0]))

MET_NMCBO = np.zeros((trials_par1, 4))
PARAMS_NMCBO = np.ones(x_init.shape)
DIST_NMCBO = np.array([]).reshape(0, dim)
RES_NMCBO = np.ones((MET_NMCBO.shape[0], x_init.shape[0]))

MET_QBO = np.zeros((trials_par1, 4))
PARAMS_QBO = np.ones(x_init.shape)
DIST_QBO = np.array([]).reshape(0, dim)
RES_QBO = np.ones((MET_QBO.shape[0], x_init.shape[0]))


   

## VP-BO 
for i, x_0 in enumerate(x_init):
    start = time.time()
    optpltvar(trials = trials_par2,
                             split_num = 2,
                             lim_init = lim_init,
                             f_cores = 3,
                             af_cores = 1,
                             ref_cores = 1,
                             x_init  = x_0)
    end = time.time()
    print('Run time '+str(end-start)+'s')
    print('iteration '+str(i+1))
    MET_VPBO1[:, 0] += time_vp.flatten()
    MET_VPBO1[:, 1] += REACOPTIM.time_fvp.flatten()
    MET_VPBO1[:, 2] += REACOPTIM.y_vpbst[:, -1].flatten()
    MET_VPBO1[:, 3] += np.min(REACOPTIM.y_vpbst[:, -1])
    print('Best VP-BO value is '+str(np.min(y_vpbst[:, -1])))
    PARAMS_VPBO1[i] = REACOPTIM.x_vp[np.argmin(REACOPTIM.y_vp[:, -1])]
    DIST_VPBO1 = np.vstack([DIST_VPBO1, REACOPTIM.x_vp])
    RES_VPBO1[:, i] = REACOPTIM.y_vpbst[:, -1]
MET_VPBO1[:, 0] = MET_VPBO1[:, 0]/(i+1)
MET_VPBO1[:, 1] = MET_VPBO1[:, 1]/(i+1)
MET_VPBO1[:, 2] = MET_VPBO1[:, 2]/(i+1)
MET_VPBO1[:, 3] = MET_VPBO1[:, 3]/(i+1)
REACOPTIM.y_vpbst = MET_VPBO1[:, 2].reshape(-1, 1)
REACOPTIM.time_vp = MET_VPBO1[:, 0].flatten()
REACOPTIM.time_fvp = MET_VPBO1[:, 1].flatten()


NameError: name 'SYST_RECYCLE' is not defined

In [ ]:
dim=2
dist_ref

In [9]:
def optpltvar(trials, split_num, liminit, scaling_factor, fcores, afcores, xinit  = None):
        """
        VP-BO approach...
        The partitions are made using the 'split_num' argument; this sets the number of blocks that the variables
        are split into. 
        The 'liminit' argument sets the initial value for the x_{-k} variables.
        Note that when introducing the design variables, x, they should be ordered by subsystem.
        For example if d = 4, and split_num = 2, introducing x = (x_a, x_b, x_c, x_d) will set
        x_1 = (x_a, x_b) and x_{-1}] = (x_c, x_d), and x_2 = (x_c, x_d) and x_{-2} = (x_a, x_b).
        The 'fcores' argument sets the number of cores used for parallel experimentation, and
        the 'afcores' argument sets the cores used for optimizing the AF
        """
        print('Partitioned Variables BO Run...')
        start = time.time()
        trialspltvar = trials
        splitvar = split_num
        timespltvar = np.zeros(trialspltvar+1)
        timefspltvar = np.zeros(trialspltvar+1)
        div = int(dim/splitvar)
        sf = scaling_factor
        refmod = dist_ref['distrefmod']

        x = liminit*np.ones((splitvar, dim))
        lwr = x.copy()
        upr = x.copy()+1e-6
        
        for i in range(splitvar):
            if xinit is None:
                x[i, i*div:(i+1)*div] = np.random.uniform(0, 1, (1, div))
            else:
                xinit = xinit.reshape(1, dim)
                x[i, i*div:(i+1)*div] = xinit[0, i*div:(i+1)*div]
            lwr[i, i*div:(i+1)*div] = 0
            upr[i, i*div:(i+1)*div] = sf
        x = np.vstack([x, liminit])
        splt = int(x.shape[0]/fcores)
        xbs = np.array(np.ones(fcores), dtype = tuple)
        
        if fcores == 1:
            xbs[0] = x
        else:
            for i in range(fcores-1):
                xbs[i] = x[i*splt:(i+1)*splt, :]
            xbs[-1] = x[(i+1)*splt:, :]

            
        startf = time.time()
        y = Parallel(n_jobs = fcores)(delayed(distmod)((start_point)) for start_point in xbs)
        if str(type(refmod))=="<class '__main__.Network'>":
            yref = Parallel(n_jobs = fcores)(delayed(refmod)(start_point) for start_point in torch.from_numpy(x).float())
            endf = time.time()
            yref = torch.hstack(yref[:]).T.reshape(-1, 1).data.numpy()
        else:
            yref = Parallel(n_jobs = fcores)(delayed(refmod)((start_point)) for start_point in xbs)
            endf = time.time()
            yref = np.vstack(yref[:])
        timefspltvar[0] = endf-startf
        y = np.vstack(y[:])
        eps = y-yref
        ybst = np.min(y, axis = 0).reshape(-1, 1).T
        modelspltvar = {}
        bndsvar = {}
        LCB = {}
        xnxt = x.copy()
        init_pts = int(round(128**(div/dim)))
        x0 = np.random.uniform(0, sf, (init_pts, dim))
        for i in range(splitvar):
            modelspltvar[str(i+1)] = gpr.GaussianProcessRegressor(kernel, alpha = 1e-6,
                                                                  n_restarts_optimizer = 10,
                                                                  normalize_y = True)
            modelspltvar[str(i+1)].fit(x, eps[:, i])
            bndsvar[str(i+1)] = Bounds(lwr[i], upr[i])
            LCB[str(i+1)] = LCB_AF(modelspltvar[str(i+1)], dim, exp_w,
                                   descale, **{'refmod': dist_ref['distrefmod'+str(i+1)]}).LCB
            opt = Parallel(n_jobs = afcores)(delayed(minimize)(LCB[str(i+1)], x0 = start_point,
                                                              method = 'L-BFGS-B',
                                                              bounds = bndsvar[str(i+1)])
                                            for start_point in x0)
            xnxts = np.array([res.x for res in opt], dtype = 'float')
            funs = np.array([np.atleast_1d(res.fun)[0] for res in opt])
            xnxt[i] = xnxts[np.argmin(funs)]
            xnxt[-1, i*div:(i+1)*div] = xnxts[np.argmin(funs), i*div:(i+1)*div]
        xnxtbs = np.array(np.ones(fcores), dtype = tuple)
        end = time.time()
        timespltvar[0] = end-start
        for i in range(trialspltvar):
            if fcores == 1:
                xnxtbs[0] = xnxt
            else:
                for j in range(fcores-1):
                    xnxtbs[j] = xnxt[j*splt:(j+1)*splt, :]
                xnxtbs[-1] = xnxt[(j+1)*splt:, :]
            startf = time.time()
            ynxt = Parallel(n_jobs = fcores)(delayed(distmod)(descale(start_point)) for start_point in xnxtbs)
            if str(type(refmod))=="<class '__main__.Network'>":
                yref = Parallel(n_jobs = fcores)(delayed(refmod)(start_point) for start_point in torch.from_numpy(x).float())
                endf = time.time()
                yref = torch.hstack(yref[:]).T.reshape(-1, 1).data.numpy()
            else:
                yref = Parallel(n_jobs = fcores)(delayed(refmod)(descale(start_point)) for start_point in xnxtbs)
                endf = time.time()
                yref = np.vstack(yref[:])
            ynxt = np.vstack(ynxt[:])
            epsnxt = ynxt-yref
            timefspltvar[i+1] = timefspltvar[i]+(endf-startf)
            for j in range(splitvar):
                if any(ynxt[:, j] < min(y[:, j])):
                    lwr[j] = xnxt[np.argmin(ynxt[:, j])]
                    lwr[j, j*div:(j+1)*div] = 0
                    upr[j] = xnxt[np.argmin(ynxt[:, j])]+1e-6
                    upr[j, j*div:(j+1)*div] = sf
            x = np.vstack([x, xnxt])
            y = np.vstack([y, ynxt])
            eps = np.vstack([eps, epsnxt])
            ybst = np.vstack([ybst, np.min(ynxt, axis = 0).reshape(-1,1).T])
            x0 = np.random.uniform(0, sf, (init_pts, dim))
            for j in range(splitvar):
                modelspltvar[str(j+1)].fit(x, eps[:, j])
                bndsvar[str(j+1)] = Bounds(lwr[j], upr[j])
                opt = Parallel(n_jobs = afcores)(delayed(minimize)(LCB[str(j+1)], x0 = start_point,
                                                                  method = 'L-BFGS-B',
                                                                  bounds = bndsvar[str(j+1)])
                                                for start_point in x0)
                xnxts = np.array([res.x for res in opt], dtype = 'float')
                funs = np.array([np.atleast_1d(res.fun)[0] for res in opt])
                xnxt[j] = xnxts[np.argmin(funs)]
                xnxt[-1, j*div:(j+1)*div] = xnxts[np.argmin(funs), j*div:(j+1)*div]
            end = time.time()
            timespltvar[i+1] = end-start
        spltvar_optim = True
        modelspltvar = modelspltvar
        xspltvar = descale(x)
        yspltvar = y
        yspltvarbst = ybst


In [ ]:
sz = x_train.shape[1]
kernel = gpr.kernels.Matern(np.ones(sz), np.array([k_bnds[i]]*sz), nu = nu[i])
dim = x_train.shape[1]


In [ ]:
def normalize(x, axis = None, mu = None, sigma = None):
    if mu is None:
        mu = np.mean(x, axis = axis)
        sigma = np.std(x, axis = axis, ddof = 1)
        return (x-mu)/sigma, mu, sigma
    
    else:
        return (x-mu)/sigma


def unnormalize(x, mu, sigma):
    return sigma*x+mu

def descale(self, x):
    m = (self.ub-self.lb)/(self.bounds.ub-self.bounds.lb)
    b = self.ub-m*self.bounds.ub
    return m*x+b


def scale(self, x, use_self = True, lb = None, ub = None):
    if use_self:    
        m = (self.bounds.ub-self.bounds.lb)/(self.ub-self.lb)
        b = self.bounds.ub-m*self.ub
    else:
        m = (self.bounds.ub-self.bounds.lb)/(ub-lb)
        b = self.bounds.ub-m*ub
    return m*x+b

if aux_mods:
    refmod = list(aux_mods.values())[0]
    if len(aux_mods) > 1:
        distmod = aux_mods['distmod']
        dist_ref['distrefmod'] = aux_mods['ref_distmod']
        for i in range(3,len(aux_mods)):
            dist_ref['distrefmod'+str(i-2)] = aux_mods['ref_distmod'+str(i-2)]
        dist_ref = OrderedDict(dist_ref)

bois_optim = False
mcbo_optim = False
opbo_optim = False

In [ ]:
def optimvpbo(trials, fcores ):
    start= time.time()
    trialspltvar = trials
    splitvar = split_num
    timespltvar= np.zeros(trialspltvar+1)
    timefspltvar = np.zeros(trialspltvar+1)
    div = int(dim/splitvar)
    sf = scaling_factor
    refmod = np.array([]).reshape(0, dim)

    x = liminit*np.ones((splitvar, dim))
    lwr = x.copy()
    upr = x.copy()+1e-6

        
    for i in range(splitvar):
        if xinit is None:
            x[i, i*div:(i+1)*div] = np.random.uniform(0, 1, (1, div))
        else:
            xinit = xinit.reshape(1, dim)
            x[i, i*div:(i+1)*div] = xinit[0, i*div:(i+1)*div]
        lwr[i, i*div:(i+1)*div] = 0
        upr[i, i*div:(i+1)*div] = sf
        x = np.vstack([x, liminit])
        splt = int(x.shape[0]/fcores)
        xbs = np.array(np.ones(fcores), dtype = tuple)


        if fcores == 1:
            xbs[0] = x
        else:
            for i in range(fcores-1):
                xbs[i] = x[i*splt:(i+1)*splt, :]
            xbs[-1] = x[(i+1)*splt:, :]

        startf = time.time()  
    y = Parallel(n_jobs = fcores)(delayed(distmod)((start_point)) for start_point in xbs)
    if str(type(refmod))=="<class '__main__.Network'>":
        yref = Parallel(n_jobs = fcores)(delayed(refmod)(start_point) for start_point in torch.from_numpy(x).float())
        endf = time.time()
        yref = torch.hstack(yref[:]).T.reshape(-1, 1).data.numpy()      
    else:
        yref = Parallel(n_jobs = fcores)(delayed(refmod)((start_point)) for start_point in xbs)
        endf = time.time()
        yref = np.vstack(yref[:])
    timefspltvar[0] = endf-startf
    y = np.vstack(y[:])
    eps = y-yref
    ybst = np.min(y, axis = 0).reshape(-1, 1).T
    modelspltvar = {}
    bndsvar = {}
    LCB = {}
    xnxt = x.copy()
    init_pts = int(round(128**(div/dim)))
    x0 = np.random.uniform(0, sf, (init_pts, dim))


    for i in range(splitvar):
        modelspltvar[str(i+1)] = gpr.GaussianProcessRegressor(kernel, alpha = 1e-6,
                                                                  n_restarts_optimizer = 10,
                                                                  normalize_y = True)
        modelspltvar[str(i+1)].fit(x, eps[:, i])
        bndsvar[str(i+1)] = Bounds(lwr[i], upr[i])
        LCB[str(i+1)] = LCB_AF(modelspltvar[str(i+1)], dim, exp_w,
                                   descale, **{'refmod': dist_ref['distrefmod'+str(i+1)]}).LCB
        opt = Parallel(n_jobs = afcores)(delayed(minimize)(LCB[str(i+1)], x0 = start_point,
                                                        method = 'L-BFGS-B',
                                                        bounds = bndsvar[str(i+1)])   
                                        for start_point in x0)   
        xnxts = np.array([res.x for res in opt], dtype = 'float')
        funs = np.array([np.atleast_1d(res.fun)[0] for res in opt])
        xnxt[i] = xnxts[np.argmin(funs)]
        xnxt[-1, i*div:(i+1)*div] = xnxts[np.argmin(funs), i*div:(i+1)*div]
    xnxtbs = np.array(np.ones(fcores), dtype = tuple)
    end = time.time()
    timespltvar[0] = end-start
    for i in range(trialspltvar):
        if fcores == 1:
            xnxtbs[0] = xnxt
        else:
            for j in range(fcores-1):
                xnxtbs[j] = xnxt[j*splt:(j+1)*splt, :]
            xnxtbs[-1] = xnxt[(j+1)*splt:, :]
        startf = time.time()
        ynxt = Parallel(n_jobs = fcores)(delayed(distmod)(descale(start_point)) for start_point in xnxtbs)
        if str(type(refmod))=="<class '__main__.Network'>":
            yref = Parallel(n_jobs = fcores)(delayed(refmod)(start_point) for start_point in torch.from_numpy(x).float())
            endf = time.time()
            yref = torch.hstack(yref[:]).T.reshape(-1, 1).data.numpy()
        else:
            yref = Parallel(n_jobs = fcores)(delayed(refmod)(descale(start_point)) for start_point in xnxtbs)
            endf = time.time()
            yref = np.vstack(yref[:])
        ynxt = np.vstack(ynxt[:])
        epsnxt = ynxt-yref
        timefspltvar[i+1] = timefspltvar[i]+(endf-startf)
        
        for j in range(splitvar):
            if any(ynxt[:, j] < min(y[:, j])):
                lwr[j] = xnxt[np.argmin(ynxt[:, j])]
                lwr[j, j*div:(j+1)*div] = 0
                upr[j] = xnxt[np.argmin(ynxt[:, j])]+1e-6
                upr[j, j*div:(j+1)*div] = sf
        x = np.vstack([x, xnxt])
        y = np.vstack([y, ynxt])
        eps = np.vstack([eps, epsnxt])
        ybst = np.vstack([ybst, np.min(ynxt, axis = 0).reshape(-1,1).T])
        x0 = np.random.uniform(0, sf, (init_pts, dim))



        for j in range(splitvar):
            modelspltvar[str(j+1)].fit(x, eps[:, j])
            bndsvar[str(j+1)] = Bounds(lwr[j], upr[j])
            opt = Parallel(n_jobs = afcores)(delayed(minimize)(LCB[str(j+1)], x0 = start_point,
                                                                method = 'L-BFGS-B',
                                                                bounds = bndsvar[str(j+1)])
                                            for start_point in x0)
            xnxts = np.array([res.x for res in opt], dtype = 'float')
            funs = np.array([np.atleast_1d(res.fun)[0] for res in opt])
            xnxt[j] = xnxts[np.argmin(funs)]
            xnxt[-1, j*div:(j+1)*div] = xnxts[np.argmin(funs), j*div:(j+1)*div]
        end = time.time()
        timespltvar[i+1] = end-start
    spltvar_optim = True
    modelspltvar = modelspltvar
    xspltvar = descale(x)
    yspltvar = y
    yspltvarbst = ybst

In [ ]:
## VP-BO 
for i, x_0 in enumerate(x_init):
    start = time.time()
    optpltvar(trials = trials_par2,
                             split_num = 2,
                             lim_init = lim_init,
                             f_cores = 3,
                             af_cores = 1,
                             ref_cores = 1,
                             x_init  = x_0)
    end = time.time()
    print('Run time '+str(end-start)+'s')
    print('iteration '+str(i+1))

In [6]:
import numpy as np

from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

from scipy.spatial.distance import euclidean

NameError: name '_C' is not defined

In [1]:
import torch
import gpytorch
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models import SingleTaskGP

# Custom function that creates a GP and mll object from data and model settings
def get_gp(
    X: torch.Tensor,
    y: torch.Tensor,
    mean_module: gpytorch.means.Mean,
    covar_module: gpytorch.kernels.Kernel,
    likelihood: gpytorch.likelihoods.Likelihood
) -> Tuple[
    gpytorch.models.GP,
    gpytorch.mlls.MarginalLogLikelihood
]:
    gp = SingleTaskGP(
        X, y,
        mean_module=mean_module,
        covar_module=covar_module,
        likelihood=likelihood
    )
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    return gp, mll

C:\Users\saadr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'Tuple' is not defined

In [ ]:
features

In [45]:
liminit= np.ones((1, 9))
liminit

array([[1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [55]:
'''FR = np.array([0.100, 0.075, 0.075])
R_Frac = 1e-6
ub = np.array([423, 423])
lb = np.array([303, 303])
lim_init = (np.array([336, 380]))
lim_init'''

array([336, 380])

In [80]:
FR = np.array([0.100, 0.075, 0.075])
R_Frac = 1e-6
ub = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1])
lb = np.array([0, 0, 0, 0, 0 , 0, 0, 0, 0])
lim_init = (np.array([0, 0, 0, 0, 0 , 0, 0, 0, 0]))
lim_init

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [81]:
bounds = Bounds((0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 1, 1,1, 1, 1,1, 1, 1))
dim = len(ub)

In [57]:
from scipy.optimize import Bounds, NonlinearConstraint


In [82]:
x_init = np.linspace(bounds.lb, bounds.ub, 5)
x_init

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
       [0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ],
       [0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ]])

In [69]:
Bounds.lb

AttributeError: type object 'Bounds' has no attribute 'lb'

In [83]:
lim_init = scale(np.array([336, 380, 300, 300, 000, 000,300,300,200]))
lim_init

array([ 0.275     ,  0.64166667, -0.025     , -0.025     , -2.525     ,
       -2.525     , -0.025     , -0.025     , -0.85833333])

In [49]:
# Train reference model (GP)
def scale(x, ub = 423, lb = 303, sf = 1):
    m = sf/(ub - lb)
    b = -lb*m
    return m*x+b

In [84]:
x = liminit*np.ones((1, 9))
x


array([[1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [85]:
x = np.vstack([x, lim_init])

In [86]:
x


array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ],
       [ 0.275     ,  0.64166667, -0.025     , -0.025     , -2.525     ,
        -2.525     , -0.025     , -0.025     , -0.85833333]])

In [87]:
init_pts = int(len(x)/2)
init_pts

1

In [88]:
refmod = np.array([]).reshape(0, dim)
refmod

array([], shape=(0, 9), dtype=float64)

In [94]:
splits = 4

In [96]:
restarts = int(round(128/(splits+1), 0))
restarts

26

In [5]:
def optim_vpbo(trials, split_num, liminit, fcores=1, afcores=1, refcores=1, x_init=x): 
    start= time.time()
    trialspltvar = trials
    splitvar = split_num
    time_vp= np.zeros(trialspltvar+1)
    time_fvp = np.zeros(trialspltvar+1)
    div = int(dim/splitvar)
    #sf = scaling_factor
    refmod = np.array([]).reshape(0, dim)
    
    x = liminit*np.ones((splitvar, dim))
    lwr = x.copy()
    upr = x.copy()+1e-6

    for i in range(splitvar):
        if x_init is None:
            x[i, i*div:(i+1)*div] = np.random.uniform(  bounds.lb,
                                                        bounds.ub,
                                                        (1, div))
        else:
            x_init = x_init.reshape(1, dim)
            x[i, i*div:(i+1)*div] = x_init[0, i*div:(i+1)*div]       
        
        lwr[i, i*div:(i+1)*div] = bounds.lb[i]
        upr[i, i*div:(i+1)*div] = bounds.ub[i]
    x = np.vstack([x, liminit])
        
    init_pts = int(len(x)/splitvar)
    splt = int(x.shape[0]/fcores)
    xbs = np.array(np.ones(fcores), dtype = tuple)

    

#results = Parallel(n_jobs=fcores)(delayed(run_trial)(params) for params in param_combinations[:trials])

    if fcores == 1:
        xbs[0] = x
    else:
        for i in range(fcores-1):
            xbs[i] = x[i*splt:(i+1)*splt, :]
        xbs[-1] = x[(i+1)*splt:, :]

    start_f =time.time()
    y= Parallel(n_jobs= fcores)(delayed(descale(x_s) for x_s in xbs))
    
    #y_ref=y_test
    
    y_test =Parallel(n_jobs = refcores)(delayed(descale(x_s) for x_s in xbs))
    y_test = np.vstack(y_test[:])
    end_f = time.time() 
    time_fvp[0] = end_f-start_f
      
    y = np.vstack(y[:])
    eps = y-y_ref
    y_best= np.min(y,azis = 0).reshape(-1, 1).T

    mll = {}
    gp = {}
    acq_fun = {}

# Simulate BO
    for i in range(splitvar):
        # 1. Create a new GP with the best setup 
        gp[str(i+1)], mll[str(i+1)] = get_gp(
            X_train,
            y_train,
            ConstantMean(),
            ScaleKernel(RBFKernel()),
            GaussianLikelihood()
        )
        
        mll[str(i+1)] = fit_gpytorch_mll(mll)
        bnds_var[str(i+1)] = Bounds(lwr[i], upr[i])
        # 2. Define and optimize an acquisition function
        acq_fun[str(i+1)] = ExpectedImprovement(model=gp[str(i+1)], 
                                                best_f=torch.min(y_train))

    restarts = int(round(128/(splits+1), 0))
    X_next = x.copy()
    X_test_best = np.array(np.ones(f_cores), dtype = tuple)

    end = time.time()

    for i in range(init_pts):
        self.time_vp[i] = (i+1)*(end-start)/init_pts
        self.time_fvp[i] = (i+1)*(end_f-start_f)/init_pts
    print('ITERATION COUNT IS AT 'f'{init_pts};\
        TOTAL ELAPSED TIME: 'f'{self.time_vp[init_pts-1]:.1f}')

    end = time.time()
    self.time_vp[0] = end-start


    for i in range(trials_vp-init_pts):
        x0 = np.random.uniform(bounds.lb,
                                   bounds.ub,
                                   (restarts, dim))
        for j in range(splitvar):
            opt = Parallel(n_jobs = af_cores)(delayed(optimize_acqf(
                                                                    acq_fun[str(j+1)],
                                                                    bounds=bounds,
                                                                    q=1, num_restarts=5,raw_samples=20,
                                                                    inequality_constraints=ineq_constraint
                                                                    )          
                                                        )
                                                  for x_0 in x0)
            
            X_test_new = np.array([res.x for res in opt], dtype = 'float')
            acqf_val = np.array([np.atleast_1d(res.fun)[0] for res in opt])
            X_test_new[j] = X_test_new[np.argmin(acqf_val)]
            X_test_new[-1, j*div:(j+1)*div] = X_test_new[np.argmin(acqf_val), j*div:(j+1)*div]

        if f_cores == 1:
                X_test_best[0] = X_test_new
        else:
                for j in range(f_cores-1):
                    X_test_best[j] = X_test_new[j*splt:(j+1)*splt, :]
                X_test_best[-1] = X_test[(j+1)*splt:, :]


        start_f = time.time()
        y_test_new = Parallel(n_jobs = f_cores)(delayed(self.distmod)(self.descale(x_s), *self.args)
                                             for x_s in x_nxtbs)



        time_fvp[i+init_pts] = time_fvp[i+init_pts-1]+(end_f-start_f)

        X_train = torch.vstack((X_train, X_test_new))
        y_train = torch.vstack((y_train, y_test_new))
        y_test_new= torch.vstack((y_test_new[:]))
        eps_next = y_test_new-y_test
        eps = torch.vstack([eps, eps_next])
        y_best = torch.vstack([y_best, np.min(y_test_new, axis = 0).reshape(-1,1).T])
        
        for j in range(splitvar):
             if any(y_test_new[:,j] < min(y[:,j])):
                lwr[j] = X_test_new[np.argmin(y_test_new[:, j])]
                lwr[j, j*div:(j+1)*div] = bounds.lb[j]
                upr[j] = X_test_new[np.argmin(y_test_new[:, j])]+1e-6
                upr[j, j*div:(j+1)*div] = bounds.ub[j]  

        for j in range(splitvar):
            mll[str(i+1)] = fit_gpytorch_mll(mll)
            acq_fun[str(i+1)] = ExpectedImprovement(model=gp[str(i+1)], 
                                                best_f=torch.min(y_train))
            [str(j+1)].fit(x, eps[:, j])
            bnds_var[str(j+1)] = Bounds(lwr[j], upr[j])          
        end =time.time()
        time_vp[i+init_pts] = end-start

        print('ITERATION COUNT IS AT 'f'{init_pts+i+1};\
                  TOTAL ELAPSED TIME: 'f'{time_vp[i+init_pts]:.1f}')
        
    vpbo_optim = True
    model_vp = gp
    x_vp = descale(x)
    y_vp = y
    y_vpbst = y_best
    


        # 3.a (Optional) Find the closes test point
        best_idx = -1
        best_dist = np.inf
        for idx, test_point in enumerate(X_test):
            cur_dist = euclidean(X_new.squeeze(), test_point)
            if cur_dist < best_dist:
                best_dist = cur_dist
                best_idx = idx

        # 3.b Update the dataset by adding the new point
        X_test_new = X_test[best_idx]
        y_test_new = y_test[best_idx]

        # Remove from test set
        X_test = torch.cat((X_test[:best_idx], X_test[best_idx + 1:]))
        y_test = torch.cat((y_test[:best_idx], y_test[best_idx + 1:]))

        print(f"Round: {i}")
        print(f" - New experimental point: {X_new}")
        print(f" - New acquisition value: {acqf_val}")
        print(f" - Test point: {X_test_new}")
        print(f" - Test evaluation: {y_test_new}")
        print(F" - Dataset shape: {X_train.shape[0]}")
        print("")

        X_train = torch.vstack((X_train, X_test_new))
        y_train = torch.vstack((y_train, y_test_new))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 101)

In [ ]:
def optim_vpbool(trials, split_num, liminit, fcores=1, afcores=1, refcores=1, x_init=x): 
    trialspltvar = trials
    splitvar = split_num
    div = int(dim/splitvar)
    #sf = scaling_factor
    
    
    x = liminit*np.ones((splitvar, dim))
    lwr = x.copy()
    upr = x.copy()+1e-6

    for i in range(splitvar):
        if x_init is None:
            x[i, i*div:(i+1)*div] = np.random.uniform(  bounds.lb,
                                                        bounds.ub,
                                                        (1, div))
        else:
            x_init = x_init.reshape(1, dim)
            x[i, i*div:(i+1)*div] = x_init[0, i*div:(i+1)*div]       
        
        lwr[i, i*div:(i+1)*div] = bounds.lb[i]
        upr[i, i*div:(i+1)*div] = bounds.ub[i]
    x = np.vstack([x, liminit])
        
    init_pts = int(len(x)/splitvar)
    splt = int(x.shape[0]/fcores)
    xbs = np.array(np.ones(fcores), dtype = tuple)

    

#results = Parallel(n_jobs=fcores)(delayed(run_trial)(params) for params in param_combinations[:trials])

    if fcores == 1:
        xbs[0] = x
    else:
        for i in range(fcores-1):
            xbs[i] = x[i*splt:(i+1)*splt, :]
        xbs[-1] = x[(i+1)*splt:, :]

    start_f =time.time()
    y= Parallel(n_jobs= fcores)(delayed(descale(x_s) for x_s in xbs))
    
    #y_ref=y_test
    
    y_test =Parallel(n_jobs = refcores)(delayed(descale(x_s) for x_s in xbs))
    y_test = np.vstack(y_test[:])
    end_f = time.time() 
    time_fvp[0] = end_f-start_f
      
    y = np.vstack(y[:])
    eps = y-y_ref
    y_best= np.min(y,azis = 0).reshape(-1, 1).T

    mll = {}
    gp = {}
    acq_fun = {}

# Simulate BO
    for i in range(splitvar):
        # 1. Create a new GP with the best setup 
        gp[str(i+1)], mll[str(i+1)] = get_gp(
            X_train,
            y_train,
            ConstantMean(),
            ScaleKernel(RBFKernel()),
            GaussianLikelihood()
        )
        
        mll[str(i+1)] = fit_gpytorch_mll(mll)
        bnds_var[str(i+1)] = Bounds(lwr[i], upr[i])
        # 2. Define and optimize an acquisition function
        acq_fun[str(i+1)] = ExpectedImprovement(model=gp[str(i+1)], 
                                                best_f=torch.min(y_train))

    restarts = int(round(128/(splits+1), 0))
    X_next = x.copy()
    X_test_best = np.array(np.ones(f_cores), dtype = tuple)

    end = time.time()

    for i in range(init_pts):
        self.time_vp[i] = (i+1)*(end-start)/init_pts
        self.time_fvp[i] = (i+1)*(end_f-start_f)/init_pts
    print('ITERATION COUNT IS AT 'f'{init_pts};\
        TOTAL ELAPSED TIME: 'f'{self.time_vp[init_pts-1]:.1f}')

    end = time.time()
    self.time_vp[0] = end-start


    for i in range(trials_vp-init_pts):
        x0 = np.random.uniform(bounds.lb,
                                   bounds.ub,
                                   (restarts, dim))
        for j in range(splitvar):
            opt = Parallel(n_jobs = af_cores)(delayed(optimize_acqf(
                                                                    acq_fun[str(j+1)],
                                                                    bounds=bounds,
                                                                    q=1, num_restarts=5,raw_samples=20,
                                                                    inequality_constraints=ineq_constraint
                                                                    )          
                                                        )
                                                  for x_0 in x0)
            
            X_test_new = np.array([res.x for res in opt], dtype = 'float')
            acqf_val = np.array([np.atleast_1d(res.fun)[0] for res in opt])
            X_test_new[j] = X_test_new[np.argmin(acqf_val)]
            X_test_new[-1, j*div:(j+1)*div] = X_test_new[np.argmin(acqf_val), j*div:(j+1)*div]

        if f_cores == 1:
                X_test_best[0] = X_test_new
        else:
                for j in range(f_cores-1):
                    X_test_best[j] = X_test_new[j*splt:(j+1)*splt, :]
                X_test_best[-1] = X_test[(j+1)*splt:, :]


        start_f = time.time()
        y_test_new = Parallel(n_jobs = f_cores)(delayed(self.distmod)(self.descale(x_s), *self.args)
                                             for x_s in x_nxtbs)



        time_fvp[i+init_pts] = time_fvp[i+init_pts-1]+(end_f-start_f)

        X_train = torch.vstack((X_train, X_test_new))
        y_train = torch.vstack((y_train, y_test_new))
        y_test_new= torch.vstack((y_test_new[:]))
        eps_next = y_test_new-y_test
        eps = torch.vstack([eps, eps_next])
        y_best = torch.vstack([y_best, np.min(y_test_new, axis = 0).reshape(-1,1).T])
        
        for j in range(splitvar):
             if any(y_test_new[:,j] < min(y[:,j])):
                lwr[j] = X_test_new[np.argmin(y_test_new[:, j])]
                lwr[j, j*div:(j+1)*div] = bounds.lb[j]
                upr[j] = X_test_new[np.argmin(y_test_new[:, j])]+1e-6
                upr[j, j*div:(j+1)*div] = bounds.ub[j]  

        for j in range(splitvar):
            mll[str(i+1)] = fit_gpytorch_mll(mll)
            acq_fun[str(i+1)] = ExpectedImprovement(model=gp[str(i+1)], 
                                                best_f=torch.min(y_train))
            [str(j+1)].fit(x, eps[:, j])
            bnds_var[str(j+1)] = Bounds(lwr[j], upr[j])          
        end =time.time()
        time_vp[i+init_pts] = end-start

        print('ITERATION COUNT IS AT 'f'{init_pts+i+1};\
                  TOTAL ELAPSED TIME: 'f'{time_vp[i+init_pts]:.1f}')
        
    vpbo_optim = True
    model_vp = gp
    x_vp = descale(x)
    y_vp = y
    y_vpbst = y_best
    


        # 3.a (Optional) Find the closes test point
        best_idx = -1
        best_dist = np.inf
        for idx, test_point in enumerate(X_test):
            cur_dist = euclidean(X_new.squeeze(), test_point)
            if cur_dist < best_dist:
                best_dist = cur_dist
                best_idx = idx

        # 3.b Update the dataset by adding the new point
        X_test_new = X_test[best_idx]
        y_test_new = y_test[best_idx]

        # Remove from test set
        X_test = torch.cat((X_test[:best_idx], X_test[best_idx + 1:]))
        y_test = torch.cat((y_test[:best_idx], y_test[best_idx + 1:]))

        print(f"Round: {i}")
        print(f" - New experimental point: {X_new}")
        print(f" - New acquisition value: {acqf_val}")
        print(f" - Test point: {X_test_new}")
        print(f" - Test evaluation: {y_test_new}")
        print(F" - Dataset shape: {X_train.shape[0]}")
        print("")

        X_train = torch.vstack((X_train, X_test_new))
        y_train = torch.vstack((y_train, y_test_new))